In [1]:
from llama_cpp import Llama
from transformers import AutoTokenizer

In [2]:
model_path='./mbti-senti-empa_model'

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = Llama(
    model_path='./mbti-senti-empa_model/unsloth.Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=1024,
    n_gpu_layers=33        # Number of model layers to offload to GPU
)

llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from ./mbti-senti-empa_model/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Mbti Senti_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                       llama.context_length u32              = 8192
llama_model_loader: - kv   6:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   7:                  llama.feed_

In [3]:
import pandas as pd

df = pd.read_csv('./empathy.csv', encoding='UTF-8')

sampled_df = df.sample(frac=0.2, random_state=42)


user0 = list(sampled_df['user0'])
user1 = list(sampled_df['user1'])

user1

['자기도 피곤하겠지만 행복한 하루 보내기를 바라.',
 '네가 그 선배를 좋아하는 만큼 불안이 클 것 같아 나도 마음이 쓰인다. 그동안 많이 고민하고 애태웠을 것 같네. 진심은 통한다고 하잖아, 네 말을 곡해해서 듣지는 않을 테니 너무 염려하지 마.',
 '그래, 중요한 건 너희가 앞으로 함께할 현재와 만들어갈 미래야. 그러니 일어나지 않을 일에 너무 연연하지 말거라. 그리고 현재에 충실하면 분명히 좋은 결과가 생길 거야.',
 '저야말로 영광이죠. 앞으로도 이렇게 좋은 일 생기면 언제든 공유해주세요.',
 '그런 일을 겪으면 누구나 크게 상처받아. 다리가 아팠던 만큼 알지도 못하면서 자기에게 뭐라고 했기 때문에 마음의 상처도 크겠지. 그 사람도 몰라서 그런 거니 당신이 이해해줘.',
 '우리 딸이 똘똘하고 현명하니 엄마 말을 네 나름대로 잘 소화한 것 같아 보기 좋구나. 그렇게 힘든 일을 이겨내며, 성장하는 모습을 보니 기특해!',
 '아이는 물론 고생한 아내의 수고를 잊지 않고 알아주는 너의 배려심이 눈부시게 두드러지는 순간이야. 두 사람이 함께한다면 아이도 걱정 없이 건강하게 잘 자랄 수 있을 거야. 다시 한번 아빠가 된 걸 축하해.',
 '어떤 사람을 만났기에 이런 반응을 보일까 너무 궁금하네.',
 '네가 힘들어 하던 그때가 생각 나는구나. 얼마나 상처가 컸으면 그 후로 다른 사람을 못 만나고 있겠니. 그 마음을 알 것 같아서 엄마도 무척 속상하네.',
 '그래 그렇게 그 상황을 이해하려 보면 화가 누그러질 거야. 좋은 감정으로 지내도 부족한 하루인데, 네 감정을 그런 부분까지 쓰지 않았으면 해. 우리 화내지 말고 그 사람들을 이해해보고 흘려 보내 버리자.',
 '나의 진심이 통해서 다행이다. 너의 문자를 받고 요즘 얼굴이 어두웠던 이유를 깨닫게 되었지. 신경 못 쓴 것 같아서 너무 미안한 마음뿐이었어.',
 '다 알고, 뻔하고, 지루한 얘기들을 몇 시간이고 들어주느라 네가 정말 지치고 열받겠구나. 엄마도 아빠에게 잡혀서 네 시간 동안 무한 반복되는

In [4]:
output_emaps = []

for text in user0:

    PROMPT = "공감하는 챗봇으로서 상대방의 입력에 대해 공감을 하자. 모든 대답은 한국어(Korean)으로 대답해줘."
    instruction = text

    messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{instruction}"}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    generation_kwargs = {
        "max_tokens": 2048,
        "stop": ["<|eot_id|>"],
        "top_p": 0.9,
        "temperature": 0.6,
        "echo": True,  # 프롬프트를 출력에 포함합니다.
    }

    response_msg = model(prompt, **generation_kwargs)
    output_empa  = response_msg['choices'][0]['text'][len(prompt):]

    output_emaps.append(output_empa)

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1129: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(

llama_print_timings:        load time =     638.71 ms
llama_print_timings:      sample time =       0.93 ms /    15 runs   (    0.06 ms per token, 16198.70 tokens per second)
llama_print_timings: prompt eval time =     638.57 ms /    78 tokens (    8.19 ms per token,   122.15 tokens per second)
llama_print_timings:        eval time =     230.27 ms /    14 runs   (   16.45 ms per token,    60.80 tokens per second)
llama_print_timings:       total time =     876.25 ms /    92 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     638.71 ms
llama_print_timings:      sample time =       2.44 ms /    45 runs   (    0.05 ms per token, 18404.91 tokens per second)
llama_print_timings: prompt eval time =      48.05 ms /    66 tokens (    0.73 ms 

In [5]:
output_emaps

['그래, 언니도 회사에서 즐겁게 지내.',
 '그냥 걱정되는 마음에 말해본 거니까 부담스럽게 생각하지 않을 거야. 네가 말하는 걸로도 상대방이 조심할 수 있는 계기가 될 수 있어.',
 '역시 우리 딸은 아버지의 말을 잘 들어. 그 마음으로 모든 일에 임하면 분명히 좋은 결과를 얻을 수 있을 거야.',
 '저도 좋은 소식을 전해줘서 기쁘네요. 그럼 다음에 또 연락할게요.',
 '와, 그거 정말 안 좋은 경험일 거 같아. 임산부에게 자리를 양보하는 게 예의가 아니라는 건 알지만, 그래도 나도 모르게 그 상황에선 그럴 수도 있잖아.',
 '이제야 마음이 편안해졌다니 다행이에요. 앞으로도 친구의 마음을 상하게 하는 일은 없었으면 좋겠어요. 힘들 때마다 엄마 조언을 생각해서 연락 주고요.',
 '당연하지. 너도 엄마를 많이 도와주고 있으니까. 엄마가 얼마나 고마워할지 상상이 가.',
 '당황스러운 마음은 충분히 이해해. 하지만 나도 경험해본 일이야. 상대방의 말에 상처받지 말고, 이 상황을 즐기려고 해봐.',
 '아이고, 우리 딸이 상처를 받았었구나. 엄마는 몰랐네. 그때 너가 얼마나 힘들었을까. 아빠와 엄마는 네가 남자친구랑 헤어진 후에 너가 너무 힘들어 보여서 우리도 많이 속상했단다.',
 '화가 풀려서 다행이야. 감정은 전염이 되기 때문에 주변 사람들도 덩달아 기분이 상할 수 있잖아. 공감을 잘하는 챗봇이 되기 위해 노력할게.',
 '이런 마음을 털어놓을 수 있는 형이 있어서 얼마나 좋은지 몰라. 네가 고민이 있을 때마다 난 언제나 네 편이란 걸 잊지 말아줘.',
 '네가 그동안 얼마나 답답하고 힘들었을지 너무 잘 알겠어! 네가 그렇게 말해도 엄마, 아빠가 못 알아들으시고 화내시는 부분이 가장 힘들었을 거야. 정말 얼마나 힘들었을지 나도 너무 잘 알겠어!',
 '네가 장래의 꿈을 위해 공부하는 것이 재미있어 자랑을 한 것인데, 친구가 너에게 상처가 되는 말을 했다니 너무 속상하겠다. 그래도 그 친구가 너를 생각해서 한 말은 아닐 거야. 너의 꿈을 진심으로 응원

In [6]:
import pickle

with open('user1.pkl', 'wb') as file1, \
    open('output_emaps.pkl', 'wb') as file2:

    pickle.dump(user1, file1)
    pickle.dump(output_emaps, file2)